## Instructions

Complete the three questions below. Each question builds on the previous one.

**Important:**
- Run cells in order from top to bottom
- Each question must save a CSV file to the `output/` directory
- Test your work locally before submitting: `pytest .github/test/test_assignment.py -v`
- Before final submission: Restart kernel and "Run All" to verify everything works

---

## Setup

Run this cell first to import libraries and create the output directory:

In [45]:
import pandas as pd
import numpy as np
import os

# Create output directory
os.makedirs('output', exist_ok=True)

print("✓ Setup complete")

✓ Setup complete


---

## Question 1: Data Loading & Exploration

**Objective:** Load the dataset, select specific columns, perform basic inspection, and generate summary statistics.

### Part A: Load and Inspect the Data

Load `data/customer_purchases.csv` and display basic information about the dataset.

In [46]:
# Load the CSV file into a DataFrame called 'df'
df = pd.read_csv('data/customer_purchases.csv')  # Replace with pd.read_csv(...)

# Display the shape of the DataFrame
# Hint: Use .shape
print("Data shape:", df.shape)


# Display the first 5 rows
# Hint: Use .head()
print("First 5 rows:\n", df.head())


# Display column names and data types
# Hint: Use .info()
print("Column names and data types:")
df.info()

Data shape: (15000, 9)
First 5 rows:
   purchase_id customer_id product_category product_name  quantity  \
0      P00001       C0147            Books        Shoes       1.0   
1      P00002       C0061    Home & Garden          Hat       1.0   
2      P00003       C0394    Home & Garden      T-Shirt       4.0   
3      P00004       C0343           Sports     Yoga Mat       2.0   
4      P00005       C0022            Books          Rug       5.0   

   price_per_item purchase_date customer_state shipping_method  
0          451.68    2024-08-12             IL       Overnight  
1          352.84    2024-03-29             WA        Standard  
2          120.66    2024-07-11             FL        Standard  
3          374.80    2024-09-08             CA        Standard  
4          311.48    2024-06-28             NY         Express  
Column names and data types:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 9 columns):
 #   Column         

### Part B: Check for Missing Values

Identify which columns have missing values and how many.

In [47]:
# Display the count of missing values for each column
# Hint: Use .isnull().sum()
print("Missing values per column:\n", df.isnull().sum())


Missing values per column:
 purchase_id           0
customer_id           0
product_category      0
product_name          0
quantity            600
price_per_item        0
purchase_date         0
customer_state        0
shipping_method     600
dtype: int64


### Part C: Select Numeric Columns

Create a new DataFrame containing only the numeric columns from the original data.

In [48]:
# Select only numeric columns into a new DataFrame called 'df_numeric'
# Hint: Use .select_dtypes(include=['number'])
df_numeric = df.select_dtypes(include=['number'])

### Part D: Generate Summary Statistics

Calculate summary statistics for the numeric columns.

In [49]:
# Generate summary statistics for df_numeric
# Hint: Use .describe()
summary_stats = df_numeric.describe()
print("Summary statistics:\n", summary_stats)

# Save the summary statistics to 'output/exploration_summary.csv'
# Hint: Use .to_csv()
# Example: summary_stats.to_csv('output/exploration_summary.csv')
summary_stats.to_csv('output/exploration_summary.csv')


Summary statistics:
            quantity  price_per_item
count  14400.000000    15000.000000
mean       3.005000      253.517283
std        1.416658      141.440121
min        1.000000       10.090000
25%        2.000000      130.582500
50%        3.000000      254.580000
75%        4.000000      375.612500
max        5.000000      499.960000


**Expected output file:** `output/exploration_summary.csv` containing summary statistics (count, mean, std, min, 25%, 50%, 75%, max) for numeric columns.

---

## Question 2: Data Cleaning & Transformation

**Objective:** Handle missing values, convert data types, filter the data, and save the cleaned dataset.

### Part A: Handle Missing Values

Fill missing values in the `quantity` column with 1, and drop rows where `shipping_method` is missing.

In [50]:
# Fill missing 'quantity' values with 1
# Hint: Use .fillna()
df['quantity'] = df['quantity'].fillna(1)

# Drop rows where 'shipping_method' is missing
# Hint: Use .dropna(subset=['column_name'])
df = df.dropna(subset=['shipping_method'])

# Verify no missing values remain in 'quantity' and 'shipping_method'
# Hint: Use .isnull().sum()
print("Missing values after cleaning:\n", df[['quantity', 'shipping_method']].isnull().sum())

Missing values after cleaning:
 quantity           0
shipping_method    0
dtype: int64


### Part B: Convert Data Types

Convert the `purchase_date` column from string to datetime, and `quantity` to integer.

In [51]:
# Convert 'purchase_date' to datetime
# Hint: Use pd.to_datetime()
df['purchase_date'] = pd.to_datetime(df['purchase_date'])

# Convert 'quantity' to integer type
# Hint: Use .astype('int64')
df['quantity'] = df['quantity'].astype('int64')

# Verify the data types changed
# Hint: Use .dtypes
print("Data types after conversion:\n", df.dtypes)

Data types after conversion:
 purchase_id                 object
customer_id                 object
product_category            object
product_name                object
quantity                     int64
price_per_item             float64
purchase_date       datetime64[ns]
customer_state              object
shipping_method             object
dtype: object


### Part C: Filter the Data

Keep only purchases from California (CA) and New York (NY) with quantity greater than or equal to 2.

In [52]:
# Filter the DataFrame to keep only:
#       - customer_state is 'CA' or 'NY'
#       - quantity >= 2
# Hint: Use boolean indexing with &
# Hint: Use .isin(['CA', 'NY']) for multiple values
df_filtered = df[(df['customer_state'].isin(['CA', 'NY'])) & (df['quantity'] >= 2)].copy()

# Display how many rows remain after filtering
print("Rows after filtering:", df_filtered.shape[0])

Rows after filtering: 3581


### Part D: Save Cleaned Data

Save the cleaned and filtered DataFrame.

In [53]:
# Save df_filtered to 'output/cleaned_data.csv' (without the index)
# Hint: Use .to_csv('filename.csv', index=False)
df_filtered.to_csv('output/cleaned_data.csv', index=False)

**Expected output file:** `output/cleaned_data.csv` containing the cleaned and filtered data (no missing values in quantity/shipping_method, only CA/NY states, quantity >= 2, datetime and integer types).

---

## Question 3: Analysis & Aggregation

**Objective:** Create calculated columns, perform groupby aggregations, find top products, and save results.

**Note:** Use the cleaned DataFrame (`df_filtered`) from Question 2 for this question.

### Part A: Create a Calculated Column

Create a new column `total_price` by multiplying `quantity` and `price_per_item`.

In [54]:
# Create 'total_price' column
# Hint: df['new_col'] = df['col1'] * df['col2']
df_filtered['total_price'] = df_filtered['quantity'] * df_filtered['price_per_item']

# Display the first few rows to verify
print("Head, total_price:\n", df_filtered.head())

Head, total_price:
    purchase_id customer_id product_category  product_name  quantity  \
3       P00004       C0343           Sports      Yoga Mat         2   
4       P00005       C0022            Books           Rug         5   
5       P00006       C0121      Electronics    Basketball         3   
11      P00012       C0444           Sports       Cushion         5   
16      P00017       C0337    Home & Garden  Garden Tools         3   

    price_per_item purchase_date customer_state shipping_method  total_price  
3           374.80    2024-09-08             CA        Standard       749.60  
4           311.48    2024-06-28             NY         Express      1557.40  
5            31.62    2024-08-28             CA       Overnight        94.86  
11           54.50    2024-09-27             CA       Overnight       272.50  
16          234.02    2024-11-04             NY         Express       702.06  


### Part B: Calculate Total Revenue by Product Category

Group the data by `product_category` and calculate the sum of `total_price` for each category.

In [55]:
# Group by 'product_category' and sum 'total_price'
# Hint: Use .groupby('column')['target'].sum()
revenue_by_category = df_filtered.groupby('product_category')['total_price'].sum()

# Sort by revenue (descending)
# Hint: Use .sort_values(ascending=False)
revenue_by_category = revenue_by_category.sort_values(ascending=False)

# Display the results
print("Revenue by product category:\n", revenue_by_category)

Revenue by product category:
 product_category
Books            670944.49
Electronics      667752.29
Home & Garden    657306.22
Clothing         616055.11
Sports           612952.69
Name: total_price, dtype: float64


### Part C: Find Top 5 Products by Quantity Sold

Find the 5 products with the highest total quantity sold.

In [56]:
# Group by 'product_name' and sum 'quantity'
top_5_products = df_filtered.groupby('product_name')['quantity'].sum()

# Get the top 5 using .nlargest(5) or .sort_values().head(5)
top_5_products = top_5_products.nlargest(5)

# Display the results
print("Top 5 products by quantity sold:\n", top_5_products)

Top 5 products by quantity sold:
 product_name
Garden Tools     655
Headphones       614
Dumbbells        604
Cushion          588
Tennis Racket    582
Name: quantity, dtype: int64


### Part D: Save Analysis Results

Combine the revenue by category and top 5 products into a summary and save it.

In [ ]:
#  Create a DataFrame with your analysis results
# Hint: You can create a DataFrame with:
#   pd.DataFrame({
#       'category_revenue': revenue_by_category,
#       'top_products': top_5_products  # You may need to reindex or align these
#   })
#
# OR save them separately and combine in a way that makes sense
# The tests will check that you saved the correct aggregated data
#
# Convert revenue_by_category and top_5_products DataFrames
rev_cat_df = revenue_by_category.reset_index()
rev_cat_df.columns = ['product_category', 'total_revenue']

top5_df = top_5_products.reset_index()
top5_df.columns = ['product_name', 'quantity_sold']

# Make top5_df the same # of rows as rev_cat_df
top5_padded = top5_df.reindex(range(len(rev_cat_df)))

# Combine into a single DataFrame
df_analysis = pd.DataFrame({
    'product_category': rev_cat_df['product_category'],
    'total_revenue': rev_cat_df['total_revenue'],
    'top_product': top5_padded['product_name'],
    'quantity_sold': top5_padded['quantity_sold']
})

# Save to 'output/analysis_results.csv'
# Make sure the CSV has at least these columns or data:
#   - Product categories with their total revenue
#   - Top products by quantity
# Format the output so the auto-grader can find the aggregated values
# Save analysis_summary or your own structured result
# analysis_summary.to_csv('output/analysis_results.csv', index=False)
df_analysis.to_csv('output/analysis_results.csv', index=False)




  product_category  total_revenue    top_product  quantity_sold
0            Books      670944.49   Garden Tools            655
1      Electronics      667752.29     Headphones            614
2    Home & Garden      657306.22      Dumbbells            604
3         Clothing      616055.11        Cushion            588
4           Sports      612952.69  Tennis Racket            582
Analysis DataFrame:
   product_category  total_revenue
0            Books      670944.49
1      Electronics      667752.29
2    Home & Garden      657306.22
3         Clothing      616055.11
4           Sports      612952.69


**Expected output file:** `output/analysis_results.csv` containing aggregated analysis results (revenue by category, top products).

In [ ]:


# TODO: Save to 'output/analysis_results.csv'
# Make sure the CSV has at least these columns or data:
#   - Product categories with their total revenue
#   - Top products by quantity
# Format the output so the auto-grader can find the aggregated values

# Example structure (adjust as needed):
analysis_summary = pd.DataFrame({
    'product_category': revenue_by_category.index,
    'total_revenue': revenue_by_category.values
})

# TODO: Save analysis_summary or your own structured result
# analysis_summary.to_csv('output/analysis_results.csv', index=False)

In [ ]:


# Convert revenue_by_category and top_5_products to DataFrames
rev_cat_df = revenue_by_category.reset_index()
rev_cat_df.columns = ['product_category', 'total_revenue']

top5_df = top_5_products.reset_index()
top5_df.columns = ['product_name', 'quantity_sold']

# Make top5_df the same number of rows as rev_cat_df by adding blank rows if needed
top5_padded = top5_df.reindex(range(len(rev_cat_df)))

# Combine into a single DataFrame
df_analysis = pd.DataFrame({
    'product_category': rev_cat_df['product_category'],
    'total_revenue': rev_cat_df['total_revenue'],
    'top_product': top5_padded['product_name'],
    'quantity_sold': top5_padded['quantity_sold']
})

# Print and save the analysis
print(df_analysis)
df_analysis.to_csv('output/analysis_results.csv', index=False)
